In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from transformers import  DistilBertModel,DistilBertTokenizerFast
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything, loggers as pl_loggers
import pandas as pd
import random
from scripts.dataset import nluDataset
from scripts.model import IC_NER
from scripts.utils import *

In [2]:
seed_everything(42)
tb_logger = pl_loggers.TensorBoardLogger('logs/')
final_slots = pd.read_csv('./data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
idx2slots  = {idx:slots for idx,slots in enumerate(final_slots)}

Global seed set to 42


In [3]:
class NLU_Dataset(pl.LightningDataModule):
    
    def __init__(self, train_dir, val_dir, tokenizer, max_len, batch_size):
        
        super().__init__()
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.max_len = max_len

    def setup(self,stage: [str] = None):
        self.train = nluDataset( self.train_dir, self.tokenizer, self.max_len,2)
        
        self.val = nluDataset( self.val_dir, self.tokenizer, self.max_len,2)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

In [4]:
class jointBert(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.IC_NER = IC_NER('distilbert-base-multilingual-cased')

    def forward(self, input_ids, attention_mask , intent_target, slots_target):
        return self.IC_NER(input_ids, attention_mask , intent_target, slots_target)

    def training_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        
        #print(batch['slots_id'], out['slot_pred'])
        self.log('train_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_IC_loss', out['ic_loss'], on_step=False, on_epoch=True, logger=True)
        self.log('train_NER_loss', out['ner_loss'], on_step=False, on_epoch=True, logger=True)
        
        return out['joint_loss']
    
    def validation_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        intent_pred, slot_pred = out['intent_pred'], out['slot_pred']
        
        self.log('val_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_IC_loss', out['ic_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_NER_loss', out['ner_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_intent_acc', accuracy(out['intent_pred'],intent_target), on_step=False, on_epoch=True,  logger=True)
        self.log('slot_f1', slot_F1(out['slot_pred'],slots_target,idx2slots), on_step=False, on_epoch=True, logger=True)
        
        
        return out['joint_loss']

    def configure_optimizers(self):
         return torch.optim.Adam(self.parameters(), lr=2e-5)

In [8]:
dm = NLU_Dataset('./data/multiATIS/splits/1_2_noise/train_EN.tsv','./data/multiATIS/splits/3_4_noise/dev_EN.tsv',
                 'distilbert-base-multilingual-cased',56,16)
model = jointBert()

In [9]:
trainer = pl.Trainer(gpus=-1,accumulate_grad_batches=4,check_val_every_n_epoch=1,logger=tb_logger)

trainer.fit(model, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type   | Params
----------------------------------
0 | IC_NER | IC_NER | 134 M 
----------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
539.478   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

c:\users\khare\miniconda3\envs\ptenv3\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)

KeyboardInterrupt



In [ ]:
tensorboard --logdir='logs/'